In [1]:
# in google colab uncomment this

#!wget https://raw.githubusercontent.com/dmitrySorokin/Practical_RL/spring19/week06_policy_based/env_batch.py
#!pip3 install gym==0.12.1
#!pip3 install -e git+https://dmsoroki@bitbucket.org/dmsoroki/gym_interf.git@master#egg=gym_interf


# XVFB will be launched if you run on a server
#import os
#if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
#    !bash ../xvfb start
#    %env DISPLAY = : 1

In [2]:
!pip3 list | grep gym

DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
gym (0.13.1)
gym-interf (0.1, /home/dmitry/dev/gym_interf)
pybulletgym (0.1, /home/dmitry/dev/pkgs/lib/pybullet-gym)


In [3]:
import gym
import gym_interf
from env_batch import ParallelEnvBatch
import numpy as np

def make_interf_env(seed):
    env = gym.make('interf-v1')
    env.set_calc_reward('delta_visib')
    #env.set_calc_image('gpu')
    env.seed(seed)
    return env


def make_env(nenvs):
    seed = list(range(nenvs))
    env = ParallelEnvBatch([
        lambda env_seed=env_seed: make_interf_env(seed=env_seed)
        for env_seed in seed
    ])
    return env


N_ENVS = 8
N_STEPS = 20
env = make_env(nenvs=N_ENVS)
obs = env.reset()
N_ACTIONS = env.action_space.n
OBS_SHAPE = obs.shape
assert obs.shape == (8, 16, 64, 64), obs.shape
assert obs.dtype == np.uint8

/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
/home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered

In [4]:
e = make_interf_env(0)
s = e.reset()
e.step(0)



((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

Next, we will need to implement a model that predicts logits and values. It is suggested that you use the same model as in [Nature DQN paper](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf) with a modification that instead of having a single output layer, it will have two output layers taking as input the output of the last hidden layer. **Note** that this model is different from the model you used in homework where you implemented DQN. You can use your favorite deep learning framework here. We suggest that you use orthogonal initialization with parameter $\sqrt{2}$ for kernels and initialize biases with zeros. 

In [5]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)


def ortho_weights(shape, scale=1.):
    """ PyTorch port of ortho_init from baselines.a2c.utils """
    shape = tuple(shape)

    if len(shape) == 2:
        flat_shape = shape[1], shape[0]
    elif len(shape) == 4:
        flat_shape = (np.prod(shape[1:]), shape[0])
    else:
        raise NotImplementedError

    a = np.random.normal(0., 1., flat_shape)
    u, _, v = np.linalg.svd(a, full_matrices=False)
    q = u if u.shape == flat_shape else v
    q = q.transpose().copy().reshape(shape)

    if len(shape) == 2:
        return torch.from_numpy((scale * q).astype(np.float32))
    if len(shape) == 4:
        return torch.from_numpy((scale * q[:, :shape[1], :shape[2]]).astype(np.float32))


def atari_initializer(module):
    """ Parameter initializer for Atari models
    Initializes Linear, Conv2d, and LSTM weights.
    """
    classname = module.__class__.__name__

    if classname == 'Linear':
        module.weight.data = ortho_weights(module.weight.data.size(), scale=np.sqrt(2.))
        module.bias.data.zero_()

    elif classname == 'Conv2d':
        module.weight.data = ortho_weights(module.weight.data.size(), scale=np.sqrt(2.))
        module.bias.data.zero_()

    elif classname == 'LSTM':
        for name, param in module.named_parameters():
            if 'weight_ih' in name:
                param.data = ortho_weights(param.data.size(), scale=1.)
            if 'weight_hh' in name:
                param.data = ortho_weights(param.data.size(), scale=1.)
            if 'bias' in name:
                param.data.zero_()
                
 
def conv2d_size_out(size, kernel_size, stride):
    """
    common use case:
    cur_layer_img_w = conv2d_size_out(cur_layer_img_w, kernel_size, stride)
    cur_layer_img_h = conv2d_size_out(cur_layer_img_h, kernel_size, stride)
    to understand the shape for dense layer's input
    """
    return (size - (kernel_size - 1) - 1) // stride  + 1


class Flatten(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x.view(x.size(0), -1)


class AtariCNN(nn.Module):
    def __init__(self, num_actions, state_shape):
        """ Basic convolutional actor-critic network for Atari 2600 games
        Equivalent to the network in the original DQN paper.
        Args:
            num_actions (int): the number of available discrete actions
        """
        super().__init__()

        self.conv = nn.Sequential(nn.Conv2d(in_channels=state_shape[1], out_channels=32, kernel_size=8, stride=4),
                                  nn.ReLU(),
                                  nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
                                  nn.ReLU(),
                                  nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
                                  nn.ReLU(),
                                  Flatten())
        
        convw = conv2d_size_out(state_shape[2], kernel_size=8, stride=4)
        convw = conv2d_size_out(convw, kernel_size=4, stride=2)
        convw = conv2d_size_out(convw, kernel_size=3, stride=1)
        
        convh = conv2d_size_out(state_shape[3], kernel_size=8, stride=4)
        convh = conv2d_size_out(convh, kernel_size=4, stride=2)
        convh = conv2d_size_out(convh, kernel_size=3, stride=1)
       
        linear_input_size = convw * convh * 64
        

        self.fc = nn.Sequential(nn.Linear(linear_input_size, 512),
                                nn.ReLU())

        self.pi = nn.Linear(512, num_actions)
        self.v = nn.Linear(512, 1)

        self.num_actions = num_actions

        # parameter initialization
        self.apply(atari_initializer)
        self.pi.weight.data = ortho_weights(self.pi.weight.size(), scale=.01)
        self.v.weight.data = ortho_weights(self.v.weight.size())

    def forward(self, x):
        """ Module forward pass
        Args:
            x (Variable): convolutional input, shaped [batch_size x 4 x 84 x 84]
        Returns:
            pi (Variable): action probability logits, shaped [batch_size x self.num_actions]
            v (Variable): value predictions, shaped [batch_size x 1]
        """
        
        conv_out = self.conv(x)

        fc_out = self.fc(conv_out)

        logits = self.pi(fc_out)
        value = self.v(fc_out)

        return logits, value



cuda


You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a dictionary of all the arrays that are needed to interact with an environment and train the model.
 Note that actions must be an `np.ndarray` while the other
tensors need to have the type determined by your deep learning framework. 

In [6]:
import gym

class ObserwationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def observation(self, state):
        return torch.tensor(state).float().to(DEVICE) / 255.0

class Wrapper(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def reward(self, rew):
      return 


env = ObserwationWrapper(env)
s = env.reset()
print(s.shape)
#s = s.transpose(0, 3, 1, 2)
#s = torch.tensor(s)
model = AtariCNN(N_ACTIONS, OBS_SHAPE).to(DEVICE)
logits, values = model(s)
print(logits)
print(values)
torch.distributions.Categorical(logits=logits).sample()

In [7]:
a = torch.tensor(np.arange(16))
a = a.reshape(4,4)
print(a)
a[:,3]
actions = torch.tensor([0,1,3,1]).reshape(-1, 1)
print('actions = ', actions)
a.gather(1, actions)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15]])
actions =  tensor([[0],
        [1],
        [3],
        [1]])


tensor([[ 0],
        [ 5],
        [11],
        [13]])

In [8]:
from torch.nn import functional as F

class Policy:
  def __init__(self, model):
    self.model = model
    
  def act(self, inputs, determ = False):
    logits, values = self.model(inputs)
    dist = torch.distributions.Categorical(logits=logits)

    if determ:
        actions = dist.probs.argmax(dim=1, keepdim=True)
    else:
        actions = dist.sample().view(-1, 1)

    log_probs = F.log_softmax(logits, dim=1)
    action_log_probs = log_probs.gather(1, actions)
    
    #print(actions.shape, logits.shape, action_log_probs.shape, values.shape)

    return {
        'actions': actions.reshape(-1).cpu().numpy(), 
        'logits': logits,
        'log_probs': action_log_probs.reshape(-1), 
        'values': values.reshape(-1)
    }
    #<Implement policy by calling model, sampling actions and computing their log probs>
    # Should return a dict containing keys ['actions', 'logits', 'log_probs', 'values'].

In [9]:
model = AtariCNN(N_ACTIONS, OBS_SHAPE).to(DEVICE)
policy = Policy(model)
s = env.reset()       
policy.act(s)

{'actions': array([1, 5, 7, 3, 1, 0, 2, 6]),
 'logits': tensor([[ 3.2460e-03,  2.2130e-04,  9.8520e-04,  2.4641e-04, -2.9515e-04,
          -3.9923e-04,  3.2959e-03, -2.1542e-03],
         [ 2.5310e-03,  1.5474e-03,  1.2582e-03,  2.7440e-04, -1.0643e-03,
          -1.7829e-03,  2.7702e-03, -1.8419e-03],
         [ 2.2369e-03,  7.3511e-04,  1.0906e-03,  5.9896e-04, -1.0082e-03,
           3.4548e-04,  2.7260e-03, -1.3125e-03],
         [ 3.3657e-03, -3.3464e-04,  5.3938e-04,  1.9687e-04,  1.2992e-04,
           1.8506e-04,  3.4589e-03, -1.7094e-03],
         [ 1.2336e-03,  4.9721e-05,  5.9780e-04,  7.1419e-04, -7.5546e-04,
           1.2508e-03,  3.1766e-03, -9.2241e-04],
         [ 2.4269e-03,  1.4068e-03,  9.2413e-04,  1.0750e-04, -8.7245e-04,
          -1.2067e-03,  2.2483e-03, -1.4429e-03],
         [ 2.6052e-03,  1.1304e-03,  8.9158e-04, -3.5806e-04, -1.2652e-03,
          -1.2548e-03,  2.9650e-03, -3.0186e-03],
         [ 2.3564e-03,  9.8325e-04,  1.2176e-03, -6.1996e-04, -1.1147e

Next will pass the environment and policy to a runner that collects partial trajectories from the environment. 
The class that does is is already implemented for you.

In [10]:
""" RL env runner """
from collections import defaultdict
import numpy as np


class EnvRunner:
  """ Reinforcement learning runner in an environment with given policy """
  def __init__(self, env, policy, nsteps,
               transforms=None, step_var=None):
    self.env = env
    self.policy = policy
    self.nsteps = nsteps
    self.transforms = transforms or []
    self.step_var = step_var if step_var is not None else 0
    self.state = {"latest_observation": self.env.reset()}

  @property
  def nenvs(self):
    """ Returns number of batched envs or `None` if env is not batched """
    return getattr(self.env.unwrapped, "nenvs", None)

  def reset(self):
    """ Resets env and runner states. """
    self.state["latest_observation"] = self.env.reset()

  def get_next(self):
    """ Runs the agent in the environment.  """
    trajectory = defaultdict(list, {"actions": []})
    observations = []
    rewards = []
    resets = []
    self.state["env_steps"] = self.nsteps

    for i in range(self.nsteps):
      observations.append(self.state["latest_observation"])
      act = self.policy.act(self.state["latest_observation"])
      if "actions" not in act:
        raise ValueError("result of policy.act must contain 'actions' "
                         f"but has keys {list(act.keys())}")
      for key, val in act.items():
        trajectory[key].append(val)

      obs, rew, done, _ = self.env.step(trajectory["actions"][-1])
      self.state["latest_observation"] = obs
      rewards.append(rew)
      resets.append(done)
      self.step_var += self.nenvs or 1

      # Only reset if the env is not batched. Batched envs should auto-reset.
      if not self.nenvs and np.all(done):
        self.state["env_steps"] = i + 1
        self.state["latest_observation"] = self.env.reset()

    trajectory.update(observations=observations, rewards=rewards, resets=resets)
    trajectory["state"] = self.state

    for transform in self.transforms:
      transform(trajectory)
    return trajectory


This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys 

* 'observations' 
* 'rewards' 
* 'resets'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment of specified length $T$ &mdash; the size of partial trajectory. 

To train the part of the model that predicts state values you will need to compute the value targets. 
Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected. 
Thus, we can implement and use `ComputeValueTargets` callable. 
The formula for the value targets is simple:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

In implementation, however, do not forget to use 
`trajectory['resets']` flags to check if you need to add the value targets at the next step when 
computing value targets for the current step. You can access `trajectory['state']['latest_observation']`
to get last observations in partial trajectory &mdash; $s_{t+T}$.

In [11]:
class ComputeValueTargets:
  def __init__(self, policy, gamma=0.99):
    self.policy = policy
    self.gamma = gamma
    
  def __call__(self, trajectory):
    value_target = policy.act(trajectory['state']['latest_observation'])['values']
    env_steps = trajectory['state']['env_steps']
    rewards = torch.tensor(trajectory['rewards'], dtype=torch.float).to(DEVICE)
    dones = torch.tensor(trajectory['resets'], dtype=torch.float).to(DEVICE)
    is_not_done = 1 - dones

    trajectory['value_targets'] = [0] * env_steps
    for i in range(env_steps):
        j = env_steps - i - 1
        value_target = rewards[j] + value_target * self.gamma * is_not_done[j]
        trajectory['value_targets'][j] = value_target


    # This method should modify trajectory inplace by adding 
    # an item with key 'value_targets' to it. 
    #<Compute value targets for a given partial trajectory>

After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `T * nenvs`, i.e. you essentially need
to flatten the first two dimensions. 

In [12]:
class MergeTimeBatch:
  """ Merges first two axes typically representing time and env batch. """
  def __call__(self, trajectory):
    # Modify trajectory inplace. 
    #<TODO: implement>
    #'actions', 'logits', 'log_probs', 'values', 'observations', 'rewards', 'resets', 'state', 'value_targets'
        
    trajectory['value_targets'] = torch.stack(trajectory['value_targets'])
    trajectory['value_targets'] = trajectory['value_targets'].flatten(0, 1)
    trajectory['values'] = torch.stack(trajectory['values']).flatten(0, 1)
    trajectory['logits'] = torch.stack(trajectory['logits']).flatten(0, 1)
    trajectory['log_probs'] = torch.stack(trajectory['log_probs']).flatten(0, 1)
    
    


In [13]:
model = AtariCNN(N_ACTIONS, OBS_SHAPE).to(DEVICE)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=N_STEPS,
                   transforms=[ComputeValueTargets(policy),
                               MergeTimeBatch()])


trajectory = runner.get_next()

Now is the time to implement the advantage actor critic algorithm itself. You can look into your lecture,
[Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and [lecture](https://www.youtube.com/watch?v=Tol_jw5hWnI&list=PLkFD6_40KJIxJMR-j5A1mkxK26gh_qg37&index=20) by Sergey Levine.

In [14]:
import torch.nn as nn

class A2C:
  def __init__(self,
               policy,
               optimizer,
               value_loss_coef=0.25,
               entropy_coef=0.01,
               max_grad_norm=0.5):
    self.policy = policy
    self.optimizer = optimizer
    self.value_loss_coef = value_loss_coef
    self.entropy_coef = entropy_coef
    self.max_grad_norm = max_grad_norm
    
    self.ploss = None
    self.vloss = None
    self.a2c_loss = None
    
    self.vtargets = None
    self.values = None
    self.advantage = None
    self.entropy = None
    
    self.grad_norm = None
    
  def policy_loss(self, trajectory):
    advantage = trajectory['value_targets'] - trajectory['values']
    
    logits = trajectory['logits']
    probs = nn.functional.softmax(logits, dim=-1)
    logprobs = nn.functional.log_softmax(logits, dim=-1)
    
    entropy = -torch.sum(probs * logprobs, dim=-1)

    actor_loss = -torch.mean(trajectory['log_probs'] * advantage.detach()) - self.entropy_coef * torch.mean(entropy)
    
    self.ploss = actor_loss.item()
    self.vtargets = torch.mean(trajectory['value_targets']).item()
    self.values = torch.mean(trajectory['values']).item()
    self.advantage = torch.mean(advantage).item()
    self.entropy = torch.mean(entropy).item()
    
    return actor_loss
 
  def value_loss(self, trajectory):
    critic_loss = torch.mean((trajectory['values'] - trajectory['value_targets'].detach())**2)
    self.vloss = critic_loss.item()
    return critic_loss
    
  def loss(self, trajectory):
    total_loss = self.policy_loss(trajectory) + self.value_loss_coef * self.value_loss(trajectory)
    self.a2c_loss = total_loss.item()
    return total_loss
      
  def step(self, trajectory):
    self.optimizer.zero_grad()
    self.loss(trajectory).backward()
    self.grad_norm = nn.utils.clip_grad_norm_(agent.parameters(), self.max_grad_norm)
    self.optimizer.step()

Now you can train your model. With reasonable hyperparameters training on a single GTX1080 for 10 million steps across all batched environments (which translates to about 5 hours of wall clock time)
it should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last 
episodes in each environment in the batch) of about 600. You should plot this quantity with respect to 
`runner.step_var` &mdash; the number of interactions with all environments. It is highly 
encouraged to also provide plots of the following quantities (these are useful for debugging as well):

* [Coefficient of Determination](https://en.wikipedia.org/wiki/Coefficient_of_determination) between 
value targets and value predictions
* Entropy of the policy $\pi$
* Value loss
* Policy loss
* Value targets
* Value predictions
* Gradient norm
* Advantages
* A2C loss

For optimization we suggest you use RMSProp with learning rate starting from 7e-4 and linearly decayed to 0, smoothing constant (alpha in PyTorch and decay in TensorFlow) equal to 0.99 and epsilon equal to 1e-5.

In [15]:
from tqdm import trange

def evaluate(policy, env, n_games=10):
    n_solved = 0
    n_steps = 0
    dist = 0
    angle = 0
    for i in range(n_games):
        s = env.reset()
        istep = 0
        while(True):
            s = s.reshape(1,16,64,64)
            action = policy.act(s, determ=True)['actions'][0]
            s, reward, done, info = env.step(action)
            istep += 1
            if done:
                n_solved += istep < 200
                n_steps += istep
                dist += info['dist']
                angle += info['angle_between_beams']
                break
    return n_solved / n_games, n_steps / n_games, dist / n_games, angle / n_games


In [16]:
%matplotlib inline

from tqdm import trange
import numpy as np
from tensorboardX import SummaryWriter

writer = SummaryWriter(
    'runs/a2c_with_visib',
    'fixed visibility calculation'
)

agent = AtariCNN(N_ACTIONS, OBS_SHAPE).to(DEVICE)
opt = torch.optim.RMSprop(agent.parameters(), lr=7e-4, alpha=0.99, eps=1e-5)
policy = Policy(agent)
evaluate_env = make_interf_env(1234)
a2c = A2C(policy, opt)
runner = EnvRunner(env, policy, nsteps=N_STEPS, transforms=[ComputeValueTargets(policy), MergeTimeBatch()])
runner.reset()

entropis = []
vlosses = []
plosses = []
vtargets = []
vpredictions = []
grad_norms = []
advantages = []
a2c_losses = []
rewards = np.zeros(N_ENVS, dtype=float)
dones = np.zeros(N_ENVS, dtype=float)
steps = np.zeros(N_ENVS, dtype=float)

eval_solved_games = []
mean_eval_steps = []
n_frames = []


for i in trange(0, int(1e7), N_ENVS * N_STEPS):
    trajectory = runner.get_next()
    a2c.step(trajectory)
    
    for batch_rewards, batch_dones in zip(trajectory['rewards'], trajectory['resets']):
        rewards += batch_rewards
        dones += batch_dones
        steps += 1
    
    entropis.append(a2c.entropy)
    vlosses.append(a2c.vloss)
    plosses.append(a2c.ploss)
    vtargets.append(a2c.vtargets)
    vpredictions.append(a2c.values)
    grad_norms.append(a2c.grad_norm)
    advantages.append(a2c.advantage)
    a2c_losses.append(a2c.a2c_loss)
        
    if np.sum(dones) >= 100:
        writer.add_scalar('entropy', np.mean(entropis), i)
        writer.add_scalar('vloss', np.mean(vlosses), i)
        writer.add_scalar('ploss', np.mean(plosses), i)
        writer.add_scalar('vtarget', np.mean(vtargets), i)
        writer.add_scalar('vprediction', np.mean(vpredictions), i)
        writer.add_scalar('grad_norm', np.mean(grad_norms), i)
        writer.add_scalar('advantage', np.mean(advantages), i)
        writer.add_scalar('loss', np.mean(a2c_losses), i)
        writer.add_scalar('entropy', np.mean(entropis), i)
        writer.add_scalar('reward', np.mean(rewards / dones), i)
        writer.add_scalar('steps', np.mean(steps / dones), i)
        
        n_solved, n_steps, dist, angle = evaluate(policy, evaluate_env)
        
        writer.add_scalar('eval_solved_games', n_solved, i)
        writer.add_scalar('eval_steps', n_steps, i)
        writer.add_scalar('dist_between_beams', dist, i)
        writer.add_scalar('angle_between_beams', angle, i)

        entropis = []
        vlosses = []
        plosses = []
        vtargets = []
        vpredictions = []
        grad_norms = []
        advantages = []
        a2c_losses = []
        rewards = np.zeros(N_ENVS, dtype=float)
        dones = np.zeros(N_ENVS, dtype=float)
        steps = np.zeros(N_ENVS, dtype=float)
        

       

 46%|████▌     | 28545/62500 [8:18:32<9:06:34,  1.04it/s] /home/dmitry/dev/gym_interf/gym_interf/envs/interf_env.py:260: RuntimeWarning: divide by zero encountered in double_scalars
  band_width_y = InterfEnv.lamb / abs(wave_vector2[1])
 68%|██████▊   | 42275/62500 [12:13:12<5:47:44,  1.03s/it] 

KeyboardInterrupt: 

In [17]:
torch.save(agent.state_dict(), 'a2c_interf_model_visib')

In [ ]:
evaluate(policy, evaluate_env, 1000)